### Homework for the Module 5 of the ML Bootcamp
#### Deploying Machine Learning Models

For reference of Commands of this course: 
   - https://github.com/DataTalksClub/machine-learning-zoomcamp/tree/master/05-deployment/workshop
   - https://github.com/amindadgar/customer-churn-app
   - https://github.com/alexeygrigorev/mlbookcamp-code/blob/master/chapter-05-deployment/churn_serving.py

__Question 1__

* Install `uv`
* What's the version of uv you installed?
* Use `--version` to find out

In [1]:
!pip install uv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 27.7 MB/s eta 0:00:00m eta 0:00:010:00:01

[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python3 -m pip install --upgrade pip


In [2]:
!uv --version

uv 0.9.5


__Initialize an empty uv project__

You should create an empty folder for homework
and do it there. 

In [3]:
!uv init

Initialized project `05-deployment`


__Question 2__

* Use uv to install Scikit-Learn version 1.6.1 
* What's the first hash for Scikit-Learn you get in the lock file?
* Include the entire string starting with sha256:, don't include quotes

In [4]:
!uv add scikit-learn==1.6.1

Using CPython 3.12.1 interpreter at: /home/codespace/.python/current/bin/python3.12
Creating virtual environment at: .venv
Resolved 6 packages in 708ms                                         
Prepared 5 packages in 1.83s                                             
░░░░░░░░░░░░░░░░░░░░ [0/5] Installing wheels...                                 warning: Failed to hardlink files; falling back to full copy. This may lead to degraded performance.
         If the cache and target directories are on different filesystems, hardlinking may not be supported.
         If this is intentional, set `export UV_LINK_MODE=copy` or use `--link-mode=copy` to suppress this warning.
Installed 5 packages in 390ms                               
 + joblib==1.5.2
 + numpy==2.3.4
 + scikit-learn==1.6.1
 + scipy==1.16.3
 + threadpoolctl==3.6.0


The first hash for Scikit-Learn you get in uv.lock: `sha256:b4fc2525eca2c69a59260f583c56a7557c6ccdf8deafdba6e060f94c1c59738e`

__Models__

We have prepared a pipeline with a dictionary vectorizer and a model.

It was trained (roughly) using this code:

```python
categorical = ['lead_source']
numeric = ['number_of_courses_viewed', 'annual_income']

df[categorical] = df[categorical].fillna('NA')
df[numeric] = df[numeric].fillna(0)

train_dict = df[categorical + numeric].to_dict(orient='records')

pipeline = make_pipeline(
    DictVectorizer(),
    LogisticRegression(solver='liblinear')
)

pipeline.fit(train_dict, y_train)
```

> **Note**: You don't need to train the model. This code is just for your reference.

And then saved with Pickle. Download it [here](https://github.com/DataTalksClub/machine-learning-zoomcamp/tree/master/cohorts/2025/05-deployment/pipeline_v1.bin).

With `wget`:

```bash
wget https://github.com/DataTalksClub/machine-learning-zoomcamp/raw/refs/heads/master/cohorts/2025/05-deployment/pipeline_v1.bin
```


In [5]:
!wget https://github.com/DataTalksClub/machine-learning-zoomcamp/raw/refs/heads/master/cohorts/2025/05-deployment/pipeline_v1.bin

--2025-10-29 01:18:25--  https://github.com/DataTalksClub/machine-learning-zoomcamp/raw/refs/heads/master/cohorts/2025/05-deployment/pipeline_v1.bin
Resolving github.com (github.com)... 20.207.73.82
Connecting to github.com (github.com)|20.207.73.82|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/DataTalksClub/machine-learning-zoomcamp/refs/heads/master/cohorts/2025/05-deployment/pipeline_v1.bin [following]
--2025-10-29 01:18:26--  https://raw.githubusercontent.com/DataTalksClub/machine-learning-zoomcamp/refs/heads/master/cohorts/2025/05-deployment/pipeline_v1.bin
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1300 (1.3K) [application/octet-stream]
Saving to: ‘pipeline_v1.bin’

pipeline_v1.bin

__Question 3__

Let's use the model!

* Write a script for loading the pipeline with pickle
* Score this record:

```json
{
    "lead_source": "paid_ads",
    "number_of_courses_viewed": 2,
    "annual_income": 79276.0
}
```

In [7]:
import pickle
with open("pipeline_v1.bin", "rb") as f_in:
    pipeline = pickle.load(f_in)
    
    record= {
        "lead_source": "paid_ads",
        "number_of_courses_viewed": 2,
        "annual_income": 79276.0
    }
    print(pipeline.predict_proba([record])[0,1])

0.5336072702798061


__Question 4__

Now let's serve this model as a web service

* Install FastAPI
* Write FastAPI code for serving the model
* Now score this client using `requests`:

```python
url = "YOUR_URL"
client = {
    "lead_source": "organic_search",
    "number_of_courses_viewed": 4,
    "annual_income": 80304.0
}
requests.post(url, json=client).json()
```

What's the probability that this client will get a subscription?

In [13]:
#!pip install fastapi uvicorn

# This method is the standard if we are not creating a virtual environment

In [14]:
!uv add fastapi uvicorn

Resolved 21 packages in 0.60ms
░░░░░░░░░░░░░░░░░░░░ [0/2] Installing wheels...                                 warning: Failed to hardlink files; falling back to full copy. This may lead to degraded performance.
         If the cache and target directories are on different filesystems, hardlinking may not be supported.
         If this is intentional, set `export UV_LINK_MODE=copy` or use `--link-mode=copy` to suppress this warning.
Installed 2 packages in 60ms                                
 + fastapi==0.120.1
 + uvicorn==0.38.0


In [15]:
!uv add --dev requests
# This is a development dependency -- we won't need it in production
# This needs to be done because we are running in virtual environment, otherwise this wasn't needed

Resolved 25 packages in 689ms                                        
Prepared 4 packages in 364ms                                             
░░░░░░░░░░░░░░░░░░░░ [0/4] Installing wheels...                                 warning: Failed to hardlink files; falling back to full copy. This may lead to degraded performance.
         If the cache and target directories are on different filesystems, hardlinking may not be supported.
         If this is intentional, set `export UV_LINK_MODE=copy` or use `--link-mode=copy` to suppress this warning.
Installed 4 packages in 13ms                                
 + certifi==2025.10.5
 + charset-normalizer==3.4.4
 + requests==2.32.5
 + urllib3==2.5.0


1. __Creating__ the file `predict.py` 
2. __Testing (Calling)__ the __webservice__ from `test.py`

1. Open Terminal
2. Go to the folder directory `cd /workspaces/datatalk_ml_zoomcamp_25/05-deployment`
3. Run `python predict.py`
4. Keep this terminal open for our next service call
5. Open New Terminal
6. Go to the folder directory `cd /workspaces/datatalk_ml_zoomcamp_25/05-deployment`
7. Run `python test.py` (This will internally call our virtual env web service through the mentioned ports)

Since instead of doing `!pip install fastapi uvicorn` , we added the packages in the virtual environment i.e. `!uv add fastapi uvicorn`\
We instead of running `python predict.py` and `python test.py` : \
We NOW run `uv run python predict.py` and `uv run python test.py`

Response is : \
{'subscribe_probability': 0.5340417283801275, 'subscribe': True} \
client is likely to get a subscription

__Docker__

Install [Docker](https://github.com/DataTalksClub/machine-learning-zoomcamp/blob/master/05-deployment/06-docker.md). 
We will use it for the next two questions.

For these questions, we prepared a base image: `agrigorev/zoomcamp-model:2025`. 
You'll need to use it (see Question 5 for an example).

This image is based on `3.13.5-slim-bookworm` and has
a pipeline with logistic regression (a different one)
as well a dictionary vectorizer inside. 

This is how the Dockerfile for this image looks like:

```docker 
FROM python:3.13.5-slim-bookworm
WORKDIR /code
COPY pipeline_v2.bin .
```

We already built it and then pushed it to [`agrigorev/zoomcamp-model:2025`](https://hub.docker.com/r/agrigorev/zoomcamp-model).

> **Note**: You don't need to build this docker image, it's just for your reference.

__Question 5__

Download the base image `agrigorev/zoomcamp-model:2025`. You can easily make it by using [docker pull](https://docs.docker.com/engine/reference/commandline/pull/) command.

So what's the size of this base image?

You can get this information when running `docker images` - it'll be in the "SIZE" column.

In [16]:
!docker image pull agrigorev/zoomcamp-model:2025

2025: Pulling from agrigorev/zoomcamp-model

2667830b: Pulling fs layer 
1082f6f0: Pulling fs layer 
f5177ae2: Pulling fs layer 
357cf7a1: Pulling fs layer 
2b9cc6ee: Pulling fs layer 
Digest: sha256:14d79fde0bbf078eb18c99c2bd007205917b758ec11060b2994963a1e485c2ae
Status: Downloaded newer image for agrigorev/zoomcamp-model:2025
docker.io/agrigorev/zoomcamp-model:2025


In [17]:
!docker images

REPOSITORY                 TAG       IMAGE ID       CREATED      SIZE
agrigorev/zoomcamp-model   2025      4a9ecc576ae9   7 days ago   121MB


__Dockerfile__

Now create your own `Dockerfile` based on the image we prepared.

It should start like that:

```docker
FROM agrigorev/zoomcamp-model:2025
# add your stuff here
```

Now complete it:

* Install all the dependencies from pyproject.toml
* Copy your FastAPI script
* Run it with uvicorn 

After that, you can build your docker image.

Create the file __Dockerfile__ which contains the docker instructions i.e. installing dependencies, copy the script file (predict.py) and finally running the web service using uvicorn

Building the container by below command:

In [19]:
!docker build -t predict-subscription .

[+] Building 0.0s (0/1)                                          docker:default
[+] Building 0.2s (2/3)                                          docker:default
 => [internal] load build definition from Dockerfile                       0.0s
 => => transferring dockerfile: 1.42kB                                     0.0s
 => [internal] load metadata for ghcr.io/astral-sh/uv:latest               0.2s
 => [internal] load metadata for docker.io/agrigorev/zoomcamp-model:2025   0.0s
[+] Building 0.2s (2/4)                                          docker:default
 => [internal] load build definition from Dockerfile                       0.0s
 => => transferring dockerfile: 1.42kB                                     0.0s
 => [internal] load metadata for ghcr.io/astral-sh/uv:latest               0.2s
 => [internal] load metadata for docker.io/agrigorev/zoomcamp-model:2025   0.0s
 => [auth] astral-sh/uv:pull token for ghcr.io                             0.0s
[+] Building 0.4s (3/4)                 

__Question 6__

Let's run your docker container!

After running it, score this client once again:

```python
url = "YOUR_URL"
client = {
    "lead_source": "organic_search",
    "number_of_courses_viewed": 4,
    "annual_income": 80304.0
}
requests.post(url, json=client).json()
```

What's the probability that this lead will convert?

Running the container by below command in the terminal: \
`docker run -it --rm -p 9696:9696 predict-subscription`

1. Dockerfile is build above
2. Open Terminal
3. Go to the folder directory cd /workspaces/datatalk_ml_zoomcamp_25/05-deployment
4. Run `docker run -it --rm -p 9696:9696 predict-subscription`
5. This starts our server in a virtual container and can be called to run the mode;
6. Keep this terminal open for our next service call
7. Open New Terminal
8. Go to the folder directory cd /workspaces/datatalk_ml_zoomcamp_25/05-deployment
9. Run `python test.py` (This will internally call our docker web service through the mentioned ports)

Response is same: \
{'subscribe_probability': 0.5340417283801275, 'subscribe': True} \
client is likely to get a subscription

Instead of calling the web-service from the function `python test.py` \
We can also do it straight with curl in the command prompt:
```json
curl -X 'POST' 'http://localhost:9696/predict' \
  -H 'accept: application/json' \
  -H 'Content-Type: application/json' \
  -d '{
    "lead_source": "organic_search",
    "number_of_courses_viewed": 4,
    "annual_income": 80304.0
}'
```
Which gives us the response of `@app.post("/predict")`

__Publishing to Docker hub__

This is just for reference, this is how we published an image to Docker hub.

`Dockerfile_base`: 

```dockerfile
FROM python:3.13.5-slim-bookworm
WORKDIR /code
COPY pipeline_v2.bin .
```

Publishing:

```bash
docker build -t mlzoomcamp2025_hw5 -f Dockerfile_base .
docker tag mlzoomcamp2025_hw5:latest agrigorev/zoomcamp-model:2025
docker push agrigorev/zoomcamp-model:2025
```